# CHL Curve Conrady Prediction

This notebook uses the 3-term Conrady model to predict the longitudinal chromatic aberration (CHL) curve based on user-defined optical and sampling conditions. The `achromatcfw.core.cfw` module is then used for real-time evaluation of color fringes through focus.

In [17]:
# Add ../src to the Python search path
import sys
from pathlib import Path
sys.path.append(str(Path('..').resolve() / 'src'))
from achromatcfw.core.cfw import Farbsaumbreite

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, FloatSlider, Checkbox, interact, Output
from IPython.display import display, clear_output

## Mathematical Model Underlying the Longitudinal-Chromatic-Shift Widget  

### Notation  

| Symbol | Units | Description |
|--------|-------|-------------|
| $\lambda$ | $\mathrm{nm}$ | generic wavelength |
| $\lambda_0$ | $\mathrm{nm}$ | design (“primary”) wavelength (slider **λ₀**) |
| $\lambda_1,\;\lambda_2$ | $\mathrm{nm}$ | anchor wavelengths that satisfy the primary-achromat condition (sliders **λ₁**, **λ₂**) |
| $f_0$ | $\mathrm{mm}$ | design focal length at $\lambda_0$ (slider **f₀**) |
| $\Phi_{0,\text{req}} = 1/f_0$ | $\mathrm{mm^{-1}}$ | required paraxial power at $\lambda_0$ |
| $a_i,\;b_i,\;c_i$ | $–,\ \mathrm{nm},\ \mathrm{nm^{3.5}}$ | Conrady dispersion coefficients for glass $i=1$ (crown) or $2$ (flint) |
| $K_i$ | $\mathrm{mm^{-1}}$ | thin-lens shape factor of element $i$ |
| $\Phi_0,\;\alpha,\;\beta$ | $\mathrm{mm^{-1}},\ \mathrm{mm^{-1}\,nm},\ \mathrm{mm^{-1}\,nm^{3.5}}$ | coefficients defined in Eq.&nbsp;(3) |
| $A,\;B,\;C$ | $\mathrm{\mu m},\ \mathrm{\mu m\,nm},\ \mathrm{\mu m\,nm^{3.5}}$ | coefficients in the longitudinal-shift series |
| $\Delta z(\lambda)$ | $\mathrm{\mu m}$ | longitudinal chromatic focal shift |

### Governing Equations  

1. **Refractive index**  
   $n_i(\lambda)=1+a_i+\dfrac{b_i}{\lambda}+\dfrac{c_i}{\lambda^{3.5}}$

2. **Element power**  
   $\Phi_i(\lambda)=\bigl[n_i(\lambda)-1\bigr]K_i$

3. **Total paraxial power of the cemented doublet**  
   $\displaystyle
     \Phi(\lambda)=
       (a_1K_1+a_2K_2)
       +\frac{b_1K_1+b_2K_2}{\lambda}
       +\frac{c_1K_1+c_2K_2}{\lambda^{3.5}}
     =\Phi_0+\frac{\alpha}{\lambda}+\frac{\beta}{\lambda^{3.5}}$

4. **Primary-achromat ratio** ($\lambda_1,\lambda_2$)  
   $\displaystyle
     r=\frac{K_2}{K_1}=-
       \frac{b_1\Delta\nu_{rb}+c_1\Delta\nu_{rb\,3.5}}
            {b_2\Delta\nu_{rb}+c_2\Delta\nu_{rb\,3.5}}$  

   where $\Delta\nu_{rb}=1/\lambda_1-1/\lambda_2$ and  
   $\Delta\nu_{rb\,3.5}=1/\lambda_1^{3.5}-1/\lambda_2^{3.5}$.

5. **Shape factors from the design power constraint** ($f_0$, $\lambda_0$)  
   $\displaystyle
     K_1=\frac{\Phi_{0,\text{req}}}
              {a_1+r a_2+\dfrac{b_1+r b_2}{\lambda_0}
                           +\dfrac{c_1+r c_2}{\lambda_0^{3.5}}},\qquad
     K_2=r\,K_1$

6. **Coefficients for longitudinal shift**  
   $A=\dfrac{1}{\Phi_0},\quad
    B=-\dfrac{\alpha}{\Phi_0^{2}},\quad
    C=-\dfrac{\beta}{\Phi_0^{2}}$

7. **Longitudinal chromatic focal shift**  
   $\displaystyle
     \Delta z(\lambda)=
       A+\frac{B}{\lambda}+\frac{C}{\lambda^{3.5}}
      -\Bigl[A+\frac{B}{\lambda_0}+\frac{C}{\lambda_0^{3.5}}\Bigr]
     =B\,\Delta\nu+C\,\Delta\nu_{3.5}$  

   with $\Delta\nu=1/\lambda-1/\lambda_0$ and  
   $\Delta\nu_{3.5}=1/\lambda^{3.5}-1/\lambda_0^{3.5}$.

### What the Widget Does  

1. Reads the glass pair and sliders ($\lambda_1,\lambda_2,\lambda_0,f_0$).  
2. Computes $r$ via Eq.&nbsp;4, then $K_1,K_2$ via Eq.&nbsp;5.  
3. Builds $\Phi_0,\alpha,\beta$ (Eq.&nbsp;3) and hence $A,B,C$ (Eq.&nbsp;6).  
4. Generates $\Delta z(\lambda)$ from both forms in Eq.&nbsp;7 and plots them; the two curves coincide to numerical precision, validating the implementation.  
5. Exports the sampled curve $\bigl[\lambda,\Delta z\bigr]$ every $10\ \mathrm{nm}$ to the global array `CHLdata_global`.


In [18]:
# -------- 0. λ-grid -------------------------------------------------------
lam_f = np.linspace(400, 700, 301)          # nm, master grid
CHLdata_global = None
out = Output()

# -------- 1. Conrady coeffs (nm domain) -----------------------------------
µm_to_nm   = 1e3
µm_to_nm35 = 1e3 ** 3.5                     # (µm)^3.5 → (nm)^3.5

glass_db = {
    "N-BK7":  dict(a=0.497482, B=1.010751e-2*µm_to_nm,
                              C=3.288943e-4*µm_to_nm35),
    "N-F2":   dict(a=0.586679, B=1.554256e-2*µm_to_nm,
                              C=1.075652e-3*µm_to_nm35),
    "N-BAK4": dict(a=0.545866, B=1.165667e-2*µm_to_nm,
                              C=4.858270e-4*µm_to_nm35),
    "N-SF5":  dict(a=0.633188, B=1.797255e-2*µm_to_nm,
                              C=1.389075e-3*µm_to_nm35),
    "N-SF6":  dict(a=0.749198, B=2.409297e-2*µm_to_nm,
                              C=2.328573e-3*µm_to_nm35),
    "N-BAF10":dict(a=0.639340, B=1.517700e-2*µm_to_nm,
                              C=7.540000e-4*µm_to_nm35),
    "SF10":   dict(a=0.680780, B=2.125000e-2*µm_to_nm,
                              C=1.759000e-3*µm_to_nm35),
}

# -------- 2. Widgets ------------------------------------------------------
g1_dd = Dropdown(options=list(glass_db), value="N-BK7", description="Glass 1")
g2_dd = Dropdown(options=list(glass_db), value="N-F2",  description="Glass 2")

f0_sl = FloatSlider(min=50, max=200, step=1, value=100,   # mm
                    description="f₀ [mm]", continuous_update=False,
                    readout_format=".0f", layout={"width": "380px"})

lam1_sl = FloatSlider(min=400, max=700, step=1, value=486.1,
                      description="λ₁ [nm]", continuous_update=False,
                      layout={"width": "380px"})
lam2_sl = FloatSlider(min=400, max=700, step=1, value=656.3,
                      description="λ₂ [nm]", continuous_update=False,
                      layout={"width": "380px"})
lam0_sl = FloatSlider(min=400, max=700, step=1, value=587.3,
                      description="λ₀ [nm]", continuous_update=False,
                      layout={"width": "380px"})
auto_chk = Checkbox(value=True, description="Auto-refocus @ λ₀")

# -------- 3. Helpers ------------------------------------------------------
def chrom_shift(lam_nm, A, B, C):
    """Δz(λ) = A + B/λ + C/λ³·⁵, λ in nm, result in µm."""
    lam_nm = np.asarray(lam_nm, dtype=np.float64)
    return A + B/lam_nm + C/lam_nm**3.5

# -------- 4. Callback -----------------------------------------------------
def update(g1, g2, f0, λ1, λ2, λ0, auto_refocus):
    global CHLdata_global
    with out:
        clear_output(wait=True)

        if abs(λ1 - λ2) < 1e-6:
            print("λ₁ and λ₂ must differ."); return

        Φ0_req = 1.0 / f0                 # mm⁻¹ (desired power at λ₀)

        # -- Conrady coefficients ----------------------------------------
        a1, b1, c1 = (glass_db[g1][k] for k in ("a", "B", "C"))
        a2, b2, c2 = (glass_db[g2][k] for k in ("a", "B", "C"))

        # -- Achromat ratio r = K₂/K₁ -----------------------------------
        Δν_rb   = 1/λ1 - 1/λ2
        Δν_rb35 = 1/λ1**3.5 - 1/λ2**3.5
        num, den = b1*Δν_rb + c1*Δν_rb35, b2*Δν_rb + c2*Δν_rb35
        if abs(den) < 1e-12:
            print("Selected glasses cannot satisfy achromatism at (λ₁,λ₂)."); return
        r = -num / den                           # Eq. (2)

        # -- Solve K₁ from f₀ condition ----------------------------------
        denom = (a1 + r*a2) + (b1 + r*b2)/λ0 + (c1 + r*c2)/λ0**3.5
        if abs(denom) < 1e-12:
            print("Division by zero in K₁ equation."); return
        K1 = Φ0_req / denom                      # Eq. (3)
        K2 = r * K1

        # -- Dispersion powers α, β --------------------------------------
        α = K1*b1 + K2*b2
        β = K1*c1 + K2*c2

        # -- Φ₀, A, B, C -------------------------------------------------
        Φ0 = a1*K1 + a2*K2                      # =Φ₀ (should equal Φ0_req numerically)
        mm_to_um = 1e3
        A =  mm_to_um / Φ0
        B = -mm_to_um * α / Φ0**2
        C = -mm_to_um * β / Φ0**2

        # -- Δz(λ) -------------------------------------------------------
        dz_direct = chrom_shift(lam_f, A, B, C)
        if auto_refocus:
            dz_direct -= chrom_shift(λ0, A, B, C)

        # analytic check: BΔν + CΔν³·⁵
        Δν   = 1/lam_f - 1/λ0
        Δν35 = 1/lam_f**3.5 - 1/λ0**3.5
        dz_diff = B*Δν + C*Δν35
        if not auto_refocus:
            dz_diff += chrom_shift(λ0, A, B, C)

        # -- sample every 10 nm -----------------------------------------
        lam_sample = np.arange(400, 701, 10)
        dz_sample  = chrom_shift(lam_sample, A, B, C)
        if auto_refocus:
            dz_sample -= chrom_shift(λ0, A, B, C)
        CHLdata_global = np.column_stack((lam_sample, dz_sample))

        # -- plotting ----------------------------------------------------
        fig, (ax_z, ax_cmp) = plt.subplots(1, 2, figsize=(12, 5),
                                           constrained_layout=True)
        ax_z.plot(lam_f, dz_direct, "r",  lw=2, label="Δz (direct)")
        ax_z.plot(lam_f, dz_diff,  "--k", lw=0.8, label="Δz (BΔν+CΔν³·⁵)")
        for lam_v in (λ1, λ2):
            ax_z.axvline(lam_v, color="k", ls=":", lw=0.5)
        if auto_refocus:
            ax_z.axvline(λ0, color="m", ls="--", lw=0.5, label="λ₀")
        ax_z.set(xlabel="λ [nm]", ylabel="Δz [µm]", xlim=(400, 700))
        ax_z.grid(lw=0.3); ax_z.legend(fontsize=8)

        ax_cmp.plot(lam_f, B/lam_f,        "g", label="B / λ")
        ax_cmp.plot(lam_f, C/lam_f**3.5,   "b", label="C / λ³·⁵")
        ax_cmp.set(xlabel="λ [nm]", ylabel="Component [µm]",
                   xlim=(400, 700)); ax_cmp.grid(lw=0.3)
        ax_cmp.legend(fontsize=8)

        tag = f" (refocus @ {λ0:.0f} nm)" if auto_refocus else ""
        fig.suptitle(
            f"{g1} + {g2}   f₀ = {f0:.1f} mm   "
            f"K₁ = {K1:+.4f}, K₂ = {K2:+.4f} (r = {r:+.4f}){tag}",
            fontsize=12)
        plt.show(); plt.close(fig)

        # -- console read-out -------------------------------------------
        print(f"Check: Φ₀(design) = {Φ0:+.4f} mm⁻¹  (should equal 1/f₀ = {Φ0_req:+.4f})")
        print(f"K₁ = {K1:+.4f}    K2 = {K2:+.4f}")
        print(f"A = {A:+.2f} µm   B = {B:+.2e} µm·nm   C = {C:+.2e} µm·nm³·⁵")
        print(f"α = {α:+.3e}   β = {β:+.3e}")
        print("CHLdata_global →", CHLdata_global.shape)

# -------- 5. Launch -------------------------------------------------------
interact(update,
         g1=g1_dd, g2=g2_dd, f0=f0_sl,
         λ1=lam1_sl, λ2=lam2_sl, λ0=lam0_sl,
         auto_refocus=auto_chk)
display(out)


interactive(children=(Dropdown(description='Glass 1', options=('N-BK7', 'N-F2', 'N-BAK4', 'N-SF5', 'N-SF6', 'N…

Output()

### Defocus-Sweep Post-Analysis  

This cell evaluates how **lateral colour blur** varies when the image
plane is stepped through a ± defocus range and reports:

* the **maximum blur width** (worst case),
* the **mean blur width** between the first and last zero crossing
  (i.e. across the in-focus zone),
* the two **zero-crossing positions** themselves.

---

#### Constants & Units  

| Name | Default | Units | Meaning |
|------|---------|-------|---------|
| `K` | 2.2 | – | lens f-number $\,K=f/\!D$ |
| `F_VALUE` | 8.0 | – | exposure-curve factor used elsewhere |
| `GAMMA_VALUE` | 1.0 | – | gamma correction factor |
| `TOL` | 0.15 | – | colour‐difference threshold for “acceptable” blur |
| `XRANGE` | 400 | $\mu\text{m}$ | half-width of the lateral-colour evaluation window |
| `defocusrange` | 1500 | $\mu\text{m}$ | half range of the longitudinal sweep |
| `defocus_step` | 10 | $\mu\text{m}$ | step size of the sweep |
| `threshold` | $10^{-6}$ | – | numerical zero for width crossing |

The defocus sweep is therefore  
$z\in[-1500,+1500]\,\mu\text{m}$ in $10\,\mu\text{m}$ increments.

---

#### Workflow  

1. **Generate defocus grid**  
   `z_vals = np.arange(-defocusrange, …, defocus_step)`

2. **Evaluate colour blur**  
   `widths[i] = Farbsaumbreite(z_i, CHLdata)`  
   (function returns the lateral colour *width* in µm at each defocus plane,
   using the sampled longitudinal-chromatic data `CHLdata_global`).

3. **Locate zero crossings**  
   Linear interpolation between successive samples where  
   $\text{width}(z)$ changes sign with respect to `threshold`.

4. **Statistics**  
   * `widths.max()` — worst blur in the sweep  
   * `mean_valid`   — mean width between first and last zero  
   * positions of the two zeros, `z0_left` and `z0_right`

5. **Diagnostic message**  
   If fewer than two zero crossings are found, the code suggests increasing
   `defocusrange`.


In [19]:
# ---------------------------------------------------------------------
# Constants (edit here, everything else updates automatically)
# ---------------------------------------------------------------------
K: float            = 2.2   # f‑number
F_VALUE: float      = 8.0   # default exposure‑curve factor
GAMMA_VALUE: float  = 1.0   # default gamma

TOL: float          = 0.15  # colour‑difference tolerance

XRANGE      = 400        # x window half width (µm)

defocusrange: int   = 1500  # ± defocus sweep range (µm)
defocus_step: int   = 10    # defocus sampling step (µm)

z_vals = np.arange(-defocusrange,
                   defocusrange + defocus_step,
                   defocus_step, dtype=float)


In [20]:
threshold = 1e-6

widths = np.array([Farbsaumbreite(z, CHLdata=CHLdata_global[:, 1]) for z in z_vals])


zero_crossings = []
for i in range(len(widths) - 1):
    w0, w1 = widths[i], widths[i + 1]
    if w0 <= threshold and w1 > threshold:
        z0 = np.interp(0.0, [w0, w1], [z_vals[i], z_vals[i + 1]])
        zero_crossings.append(z0)
    elif w0 > threshold and w1 <= threshold:
        z0 = np.interp(0.0, [w0, w1], [z_vals[i], z_vals[i + 1]])
        zero_crossings.append(z0)

if len(zero_crossings) >= 2:
    z0_left, z0_right = zero_crossings[0], zero_crossings[-1]

    mask = (z_vals >= z0_left) & (z_vals <= z0_right)
    mean_valid = widths[mask].mean()

    print(f"Max  width: {widths.max():.2f}")
    print(f"Mean width (between first & last zero): {mean_valid:.2f}")
    print(f"First zero at  z ≈ {z0_left:.3f}    " f"Last zero at  z ≈ {z0_right:.3f}")
    print("To make the computation faster, use a narrower defocus range.")

else:
    print("Please enlarge the defocus range to find zero crossings.")


Max  width: 119.00
Mean width (between first & last zero): 86.32
First zero at  z ≈ -910.000    Last zero at  z ≈ 1010.000
To make the computation faster, use a narrower defocus range.
